In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Sanju\\WORKSPACE\\Late-Delivery-Classification-Machine-Learning-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Sanju\\WORKSPACE\\Late-Delivery-Classification-Machine-Learning-Project'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/SanjuShusanth/Late-Delivery-Classification-Machine-Learning-Project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'SanjuShusanth'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '21247aca5fdb09024d892f1869f14d7aab91d573'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evalution_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XgBoostClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/SanjuShusanth/Late-Delivery-Classification-Machine-Learning-Project.mlflow",
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import  accuracy_score, precision_score, recall_score, roc_auc_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from MLProject.utils.common import save_json

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)

        return accuracy, precision, recall
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        test_data = test_data.dropna(subset=[self.config.target_column])
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_quantities = model.predict(test_x)
            (accuracy, precision, recall) = self.eval_metrics(test_y, predicted_quantities)

            # saving metric as local

            scores = {'accuracy': accuracy, 'precision': precision, 'recall':recall}
            save_json(path= Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric('accuracy', accuracy)
            mlflow.log_metric('precision', precision)
            mlflow.log_metric('recall', recall)

            if tracking_url_type_store != 'file':


                mlflow.sklearn.log_model(model, 'model', registered_model_name='XgboostClassificationModel')

            else:
                mlflow.sklearn.log_model(model, 'model')

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evalution_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-29 22:37:23,094: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-29 22:37:23,098: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-29 22:37:23,101: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-29 22:37:23,103: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'xgboostclassifier'"